# OpenSearch 벡터 스토어 및 Amazon Bedrock를 사용한 RAG 
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.
---

OpenSearch는 대규모 데이터셋에 대한 유사도 검색을 위한 강력한 엔진입니다. Amazon OpenSearch Service를 통해 쉽게 클라우드 환경에서도 이용할 수 있습니다. 이와 함께 Vector Store를 사용하면 고차원 벡터 데이터를 효율적으로 저장하고 빠르게 검색할 수 있어, 복잡한 자연어 처리 작업을 더욱 간편하게 수행할 수 있습니다.


In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

no stored variable or alias opensearch_user_id
no stored variable or alias opensearch_user_password
no stored variable or alias domain_name
no stored variable or alias opensearch_domain_endpoint
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models(verbose=True))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
[{'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'inputModalities': ['TEXT'],
  'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
  'modelId': 'amazon.titan-tg1-large',
  'modelLifecycle': {'status': 'ACTIVE'},
  'modelName': 'Titan Text Large',
  'outputModalities': ['TEXT'],
  'providerName': 'Amazon',
  'responseStreamingSupported': True},
 {'customizationsSupported': ['FINE_TUNING'],
  'inferenceTypesSupported': ['ON_DEMAND', 'PROVISIONED'],
  'inputModalities': ['TEXT', 'IMAGE'],
  'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
  'modelId': 'amazon.titan-image-generator-v1:0',
  'modelLifecycle': {'status': 'ACTIVE'},
  'modelName': 'Titan Image Generator G1',
  'outputModalities': ['IMAGE'],


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2-1)

In [6]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [7]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f210c02f610>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f2135838b50>])

## Embedding 모델 선택

In [9]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )

        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [10]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)     

Bedrock Embeddings Model Loaded


# 3. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [11]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [12]:
def preprocess_data(df):
    ldf = df.copy()

    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    # df_index.to_csv("rag_data_kr/fsi_smart_faq_ko_answer.csv", index=None, header=None)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [13]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [14]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)


## 메타 데이터 생성
- 기존 Category 컬럼을 ask 로 변경 합니다.
- 컬럼의 type, source 는 metadata 로 생성하고, 내용에서는 삭제 합니다.
- 타임스탬프 및 임베딩 모델의 엔드포인트 이름을 metadata 로 추가 합니다.

In [15]:
import time
documents_fsi = loader.load()

def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        # print("stype: ", stype)
        split_content = doc.page_content.split("type: ")
        content = split_content[0]        
        metadata = split_content[1]                
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        # doc.metadata['type'] = doc.page_content.split("type: ")[1].split("\n")[0]
        doc.metadata['timestamp'] = time.time()

        
create_metadata(documents_fsi)


In [16]:
print(len(documents_fsi))

documents_fsi[0]
# documents_fsi[-1]

92


Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1702378636.2238264})

## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [17]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 0
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0
    

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


## 청킹 통계 및 내용 확인

In [18]:
from utils.rag import show_context_used, show_chunk_stat
    
show_chunk_stat(docs)

                 0
count    92.000000
mean    313.108696
std     176.256460
min      67.000000
25%     185.250000
50%     270.000000
75%     403.750000
max    1187.000000
Average length among 92 documents loaded is 313 characters.

Show document at maximum size
ask: 해외고객 추가인증 방법
Information: 먼저 이용에 불편을 드려 대단히 죄송합니다. 2013년 9월 전자금융사기 예방서비스 전면 시행으로 인해, 국내외에서 뱅킹이용시 거래에 따라 추가인증을 거치게 되었습니다.
① 인증서 발급/재발급, 1일 누적 300만원 이상 이체시 시행하는 추가인증 방법(누적금액은 변경될수 있습니다.)해외IP로 접속하신 부분이 자동으로 확인되면 ‘해외체류 확인’ 항목이 활성화 되기 때문에 법무부 출입국관리사무소에 출국정보를 확인 받는 부분에 동의하시면 ARS 인증없이 거래가 가능합니다.
※ 해외 소재한 국내기업의 내부 인터넷망을 이용하시는 경우 국내IP로 인식하여 해당 추가 인증기능 활성화가 안되는 경우가 있으니 유의부탁드립니다.
② 해외에서 온라인을 통한 예금해지/대출실행 업무시 추가인증 방법한국시간 기준으로 은행 영업일의 오전9시부터 오후6시 사이에 뱅킹에 접속하시면  해외 현지 연락처를 직접 입력하여 추가인증의 진행이 가능하도록 되어있습니다. 
③ 고객정보 보호 목적으로 인터넷뱅킹상에서 개인정보 변경 등의 일부 중요업무이 경우는 ARS인증만을 시행하도록 하고 있습니다.  왜냐하면, 타인에 의한 정보변경 시도 등을 차단하고자 하는 부분이므로 만약 해외에서 070 등 인터넷전화를 사용하시는 경우 미리 한국에서 정보변경을 하지 못하셨다면 고객센터 82-2-3449-8000 또는 82-2-3708-8000으로 상담사에게 문의하시기 바랍니다. 
④ “불법거래 방지를 위

In [19]:
show_context_used(docs, limit=5)

-----------------------------------------------
1. Chunk: 300 Characters
-----------------------------------------------
ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.
metadata:
 {'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1702378636.2238264}
-----------------------------------------------
2. Chunk: 520 Characters
-----------------------------------------------
ask: 아마존 공동인증서와 금융인증서 차이점이 무엇인가요?
Information: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다.
아마존 공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.
인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수
있습니다.

아마존 금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.
금융결제원 클라우드에 

# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 20_applications/02_qa_chatbot/01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [20]:
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
index_name = "genai-demo-index-v1"

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [21]:
from utils.opensearch import opensearch_utils

In [22]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )
else:
    print("Index does not exist")

index_name=genai-demo-index-v1, exists=True

Deleting index:
{'acknowledged': True}


In [23]:
from langchain.vectorstores import OpenSearchVectorSearch

In [24]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

CPU times: user 262 ms, sys: 14 ms, total: 276 ms
Wall time: 9.86 s


## 인덱싱된 벡터 확인

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [25]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Dashboard 통한 Query 및 레코드 확인
- OpenSearch Dashboards URL 은 Amazon OpenSearch Servie 콘솔에 가시면 확인할 수 있습니다.
OpenSearch DevTools Console DSL 방법
- [OpenSearch_Query_DSL](https://opensearch.org/docs/latest/query-dsl/index/)

![record_opensearch](img/record_opensearch.jpg)

# 5.오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [26]:
from langchain.chains.question_answering import load_qa_chain

In [27]:
def get_semantic_similar_docs(**kwargs):

    search_types = ["approximate_search", "script_scoring", "painless_scripting"]
    space_types = ["l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit"]

    assert "vector_db" in kwargs, "Check your vector_db"
    assert "query" in kwargs, "Check your query"
    assert kwargs.get("search_type", "approximate_search") in search_types, f'Check your search_type: {search_types}'
    assert kwargs.get("space_type", "l2") in space_types, f'Check your space_type: {space_types}'

    results = kwargs["vector_db"].similarity_search_with_score(
            query=kwargs["query"],
            k=kwargs.get("k", 5),
            search_type=kwargs.get("search_type", "approximate_search"),
            space_type=kwargs.get("space_type", "l2"),
            #boolean_filter=kwargs.get("boolean_filter", {}),        
            boolean_filter=opensearch_utils.get_filter(
                filter=kwargs.get("boolean_filter", [])
            ),        
            # fetch_k=3,
        )

    print ("\nsemantic search args: ")
    pprint ({
        "k": kwargs.get("k", 5),
        "search_type": kwargs.get("search_type", "approximate_search"),
        "space_type": kwargs.get("space_type", "l2"),
        "boolean_filter": opensearch_utils.get_filter(filter=kwargs.get("boolean_filter", []))
    })
    
    if kwargs.get("hybrid", False) and results:            
        max_score = results[0][1]
        new_results = []
        for doc in results:
            nomalized_score = float(doc[1]/max_score)
            new_results.append((doc[0], nomalized_score))
        results = copy.deepcopy(new_results)

    return results

## Filter 없이 실행. 

In [28]:
query = '간편조회서비스는 회원가입해야하나요?'
#query = "타기관OTP 등록 방법 알려주세요"

pre_similar_doc = get_semantic_similar_docs(
    vector_db=vector_db,
    query=query,
    k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)


semantic search args: 
{'boolean_filter': {'bool': {'filter': []}},
 'k': 3,
 'search_type': 'approximate_search',
 'space_type': 'l2'}

Score: 0.005994044
Document Number: 20
ask: 간편조회서비스는 회원가입해야만 이용할 수 있나요?
Information: 간편조회서비스에는 로그인을 위해 회원가입이 필요한 서비스와 회원가입 없이 누구나 이용 가능한 서비스가 있습니다.
Metadata:
Type: 간편서비스
Source: 신한은행
--------------------------------------------------

Score: 0.0048865345
Document Number: 49
ask: 홈페이지 회원가입은 어떻게 해야 하나요?
Information: 홈페이지 상단에 있는 [고객센터]를 클릭하여, 회원서비스 → 회원가입메뉴에서 회원가입 메뉴를 이용하시기 바랍니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
Metadata:
Type: 홈페이지
Source: 신한은행
--------------------------------------------------

Score: 0.0047204113
Document Number: 19
ask: 아이핀으로 홈페이지회원 가입한 고객은 간편조회서비스 이용할 수 없나요?
Information: 아이핀으로 홈페이지 회원가입하신 고객은 홈페이지에서 개인회원으로 전환 후 간편조회서비스 이용가능 합니다. 
Metadata:
Type: 간편서비스
Source: 신한은행
--------------------------------------------------


## metadata의 type, source 에 필터를 걸어서 검색

In [29]:
filter01 = "인터넷뱅킹"
#filter01 = "인증서"
filter02 = "신한은행"
#filter02 = "아마존은행"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

pre_similar_doc = get_semantic_similar_docs(
    vector_db=vector_db,
    query=query,
    boolean_filter=boolean_filter,
    k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)


semantic search args: 
{'boolean_filter': {'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}},
                                        {'term': {'metadata.source': '신한은행'}}]}},
 'k': 3,
 'search_type': 'approximate_search',
 'space_type': 'l2'}

Score: 0.004297656
Document Number: 91
ask: 인터넷뱅킹을 이용하려면 어떻게 해야 합니까?
Information: 영업점에 방문하지 않아도, 모바일에서 신한 쏠 (SOL) 어플 다운로드후 회원가입하시면 인터넷뱅킹/모바일뱅킹 가입이 가능합니다. ※ 만 14세 미만의 경우 법정대리인(부모/친권자/후견인)에 의해 영업점 방문하여 가입가능함 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.0039641173
Document Number: 11
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.0038381089
Document Number: 68
ask: 인터넷뱅킹 자동이체해지 방법안내
Information: 인터넷뱅킹에 로그인하신후, 좌측의 메뉴항목에서 이체 > 자동이체 > 자동이체조회 변경/취소 서비스를 통해 해지하실 수 있습니다.
Metadata:
Type: 인

# 6.사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [30]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [31]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

def run_RetrievalQA(**kwargs):

    chain_types = ["stuff", "map_reduce", "refine"]

    assert "llm" in kwargs, "Check your llm"
    assert "query" in kwargs, "Check your query"
    assert "prompt" in kwargs, "Check your prompt"
    assert "vector_db" in kwargs, "Check your vector_db"
    assert kwargs.get("chain_type", "stuff") in chain_types, f'Check your chain_type, {chain_types}'

    qa = RetrievalQA.from_chain_type(
        llm=kwargs["llm"],
        chain_type=kwargs.get("chain_type", "stuff"),
        retriever=kwargs["vector_db"].as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": kwargs.get("k", 5),
                "boolean_filter": opensearch_utils.get_filter(
                    filter=kwargs.get("boolean_filter", [])
                ),
            }
        ),
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": kwargs["prompt"],
            "verbose": kwargs.get("verbose", False),
        },
        verbose=kwargs.get("verbose", False)
    )

    return qa(kwargs["query"])

In [32]:
from utils.rag import run_RetrievalQA

In [33]:
#query = '간편조회서비스는 회원가입해야하나요?'
#query = "타기관OTP 등록 방법 알려주세요"
query = "홈페이지 탈퇴하는방법 알려줘"

result = run_RetrievalQA(
    query=query,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])

 정보에 따르면, 홈페이지 탈퇴하는 방법은 다음과 같습니다:

홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴 진행할 수 있습니다. 

혹시, 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해서 비밀번호 재설정하고 로그인 후 회원탈퇴 가능합니다.

기타 문의는 신한은행고객센터 (☎ 1599-8000번)으로 문의 바랍니다.##################################
query:  홈페이지 탈퇴하는방법 알려줘
##################################
 정보에 따르면, 홈페이지 탈퇴하는 방법은 다음과 같습니다:

홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴 진행할 수 있습니다.

혹시, 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해서 비밀번호 재설정하고 로그인 후 회원탈퇴 가능합니다.

기타 문의는 신한은행고객센터 (☎ 1599-8000번)으로 문의 바랍니다.


In [34]:
show_context_used(result['source_documents'])

-----------------------------------------------
1. Chunk: 225 Characters
-----------------------------------------------
ask: 홈페이지 고객정보를 수정하고 싶습니다.
Information: 홈페이지 회원로그인 후 정보수정을 하실 수 있습니다. 홈페이지 상단에 있는 [고객센터]를 클릭하여, 회원서비스 → 회원정보변경 메뉴에서 회원정보변경을
이용하시기 바랍니다. 인터넷뱅킹의 회원정보는 인터넷뱅킹으로 로그인 하신 후 사용자 관리메뉴를 이용하여 수정하시기 바랍니다. 기타 문의는 콜센터 1599-8000번으로 문의
바랍니다.
metadata:
 {'source': '신한은행', 'row': 46, 'type': '홈페이지', 'timestamp': 1702378636.2239668}
-----------------------------------------------
2. Chunk: 172 Characters
-----------------------------------------------
ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는
사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 45, 'type': '홈페이지', 'timestamp': 1702378636.2239642}
-----------------------------------------------
3. Chunk: 245 Characters
-----------------------------------------------
ask: 홈페이지 탈회하고 싶습니다. 어떻게 하나요?
Information: 홈페이지 로그인 후 회원탈퇴

In [35]:
query = "홈페이지 탈퇴하는방법 알려줘"

filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=False,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")
print_ww(result['result'])

 네, 홈페이지 탈퇴하는 방법은 다음과 같습니다:

홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴를 진행할 수 있습니다. 

혹시 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해 비밀번호를 재설정한 후 로그인하시고 회원탈퇴를 하시면 됩니다.

기타 문의사항은 신한은행 고객센터(☎1599-8000번)으로 문의하시기 바랍니다.##################################
query:  홈페이지 탈퇴하는방법 알려줘
boolean_filter:  [{'term': {'metadata.type': '홈페이지'}}, {'term': {'metadata.source': '신한은행'}}]
##################################
 네, 홈페이지 탈퇴하는 방법은 다음과 같습니다:

홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴를 진행할 수 있습니다.

혹시 비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해 비밀번호를 재설정한 후 로그인하시고 회원탈퇴를 하시면 됩니다.

기타 문의사항은 신한은행 고객센터(☎1599-8000번)으로 문의하시기 바랍니다.


In [36]:
show_context_used(result['source_documents'])

-----------------------------------------------
1. Chunk: 225 Characters
-----------------------------------------------
ask: 홈페이지 고객정보를 수정하고 싶습니다.
Information: 홈페이지 회원로그인 후 정보수정을 하실 수 있습니다. 홈페이지 상단에 있는 [고객센터]를 클릭하여, 회원서비스 → 회원정보변경 메뉴에서 회원정보변경을
이용하시기 바랍니다. 인터넷뱅킹의 회원정보는 인터넷뱅킹으로 로그인 하신 후 사용자 관리메뉴를 이용하여 수정하시기 바랍니다. 기타 문의는 콜센터 1599-8000번으로 문의
바랍니다.
metadata:
 {'source': '신한은행', 'row': 46, 'type': '홈페이지', 'timestamp': 1702378636.2239668}
-----------------------------------------------
2. Chunk: 172 Characters
-----------------------------------------------
ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는
사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 45, 'type': '홈페이지', 'timestamp': 1702378636.2239642}
-----------------------------------------------
3. Chunk: 245 Characters
-----------------------------------------------
ask: 홈페이지 탈회하고 싶습니다. 어떻게 하나요?
Information: 홈페이지 로그인 후 회원탈퇴

In [37]:
query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
#filter02 = "아마존은행"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=True,
    k=3
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")
print_ww(result['result'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다. 
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다. 
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는 사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.

ask: 만14세 미만의 고객은 홈페이지 회원가입이 가능하나요?
Information: 만 14세 미만 고객은 회원가입시 '정보통신망 이용촉진 및 정보 등에 관한 법률' 및 '개인정보보호지침'에 따라 법정대리인의 정보활용 동의가 필요합니다. 당행 영업점을 통한 보호자분의 홈페이지 회원 법정대리인 등록이 완료된 고객님에 한하여 회원등록이 가능하므로 홈페이지 회원 법정대리인 등록이 안된 고객님의 보호자분께서는 먼저 가까운 영업점을 방문하여 주시기 바랍니다.

In [38]:
show_context_used(result['source_documents'])

-----------------------------------------------
1. Chunk: 232 Characters
-----------------------------------------------
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다.
기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 21, 'type': '홈페이지', 'timestamp': 1702378636.2238896}
-----------------------------------------------
2. Chunk: 172 Characters
-----------------------------------------------
ask: 홈페이지 회원아이디를 변경하고 싶습니다.
Information: 회원탈퇴를 하시고, 사용하시고 싶은 아이디로 신규 회원가입신청을 하시면 됩니다. 다만, 사용하시고 싶은 아이디가 이미 다른 고객이 사용하고 있는 경우에는
사용하실 수 없습니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 45, 'type': '홈페이지', 'timestamp': 1702378636.2239642}
-----------------------------------------------
3. Chunk: 436 Characters
-----------------------------------------------
ask: 만14세 미만의 고객은 홈페이지 회원가입이 가능하나요?
Information: 만 14세 